Ahh, statistical hypothesis testing—it's a cornerstone of classical statistical inference. 
It can also seem a bit daunting when presented as a list of [over 100 different methods](https://en.wikipedia.org/wiki/Category:Statistical_tests) 
from which the user is expected to choose the right procedure based on their application domain, data, and question.
But it doesn't have to be that way

*It turns out that many traditional frequentist tests can be viewed as special cases of regression models.*

This is a powerful and liberating idea, because it can help us transition from choosing from the zoo of canned methods to building our own bespoke analyses, tailored to our exact use case.
Adopting this perspective will allow us to

- transcend rote memorization of which test should be applied where and instead reason from first principles
- make our assumptions clear and explicit 
- make extensions like incorporating covariates and interactions more natural

So the plan for this post is to take a look at a few of the most common statistical tests, and formulate each one as a regression model. We'll implement each one using its dedicated stats library method and also a generic regression model.

Let's roll!